# Group 11 DL Project - Part A 

## Imports 

In [32]:

!pip install nltk
import gensim
import numpy as np
import pandas as pd
import sklearn as sk
import torch
import torch.nn as nn
import torch.nn.functional as F

from os import listdir
from os.path import isfile, join
import re
import nltk
from nltk.corpus import stopwords

from typing import Optional

import gensim.downloader as api

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.


**Preprocessing - tokenization**

In [33]:

def comments_tokenization(dir_path, bag_of_words):
    #get all the files from the directory
    dir_file_path = dir_path
    onlyfiles = [f for f in listdir(dir_file_path) if isfile(join(dir_file_path, f))]
    files_paths = [dir_file_path + "\\" + f for f in onlyfiles]
    
    comments_tokens = [] #[[tokens_from_1_file],[tokens_from_2_file]] e.t.c
    regex = re.compile('[^a-zA-Z]') #regex for removing all the symbols that are not letters

    #for each comment (=file)
    for path_to_file in files_paths:
        with open(path_to_file, encoding="utf8") as f:
            word_data = f.read() #the data from file with comments
            words = word_data.split() 
            clean_list = [] #this list will contain only words that doesn't have symbols other than letters

            #for each words in comment (=file)
            for w in words:
                clean_word_data = regex.sub('', w) #clean the word by regex
                if clean_word_data != " " and len(clean_word_data) > 0:
                    #cleaning the stopwords from the sentence
                    clean_word_data = clean_word_data.lower()
                    #checking that the word is not a stop word (=a word that shows up frequently)
                    if clean_word_data not in en_stops:
                        clean_list.append(clean_word_data) #creates a clean comment
                        bag_of_words.append(clean_word_data)
        comments_tokens.append(clean_list) 

    return comments_tokens, bag_of_words

en_stops = set(stopwords.words('english'))
bag_of_words = []
bag_of_words_neg = []
## Positive comments tokenization ##
pos_dir_path = "\\data\\notebook_files\\train\\pos"
pos_tokens_list, bag_of_words = comments_tokenization(pos_dir_path, bag_of_words)
## Negative comments tokenization ##
neg_dir_path = "\\data\\notebook_files\\train\\neg"
neg_tokens_list, bag_of_words_neg = comments_tokenization(neg_dir_path, bag_of_words_neg)

#Creating one bag of words in order to add new words to the embeddings
for bag in bag_of_words_neg:
    if bag not in bag_of_words:
        bag_of_words.append(bag)


LookupError: LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/datalore/nltk_data'
    - '/opt/python/envs/default/nltk_data'
    - '/opt/python/envs/default/share/nltk_data'
    - '/opt/python/envs/default/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Pre-Processing - Embeddings 

First, we will enable Torch Embedding Module & load pre-trained Embeddings 

In [17]:
info = api.info()


# from gensim.models import FastText
# sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
# model = FastText(vector_size=4, window=3, min_count=1)

### Loading Embedding models (Gilad)

In [18]:
# word2vec_model = api.load('word2vec-google-news-300') # loading the word2vec embedding model from the gensim
# glove_model = api.load('glove-wiki-gigaword-300') # loading the GLoVE embedding model from the gensim

## אם תרצה להוריד ממש, אז שחרר את הגרסה למטה באפור
# glove_path = api.load('glove-wiki-gigaword-300', return_path=True)  # If we want to save the model to a local file
# print(glove_path)
# try:
#     glove_model = gensim.models.keyedvectors.load_word2vec_format(glove_path)
# except:
#     glove_model = gensim.models.keyedvectors.load_word2vec_format(glove_path, binary=True)

embedding_models=[glove_model]

embed_model=embedding_models[0]

Load the model to torch Embedding

In [19]:
weights = torch.FloatTensor(embed_model.vectors)
embedding = nn.Embedding.from_pretrained(weights)
embedding.requires_grad_(False)
print(embedding)

for index, word in enumerate(embed_model.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(embed_model.index_to_key)} is {word}")

Embedding(400000, 300)
word #0/400000 is the
word #1/400000 is ,
word #2/400000 is .
word #3/400000 is of
word #4/400000 is to
word #5/400000 is and
word #6/400000 is in
word #7/400000 is a
word #8/400000 is "
word #9/400000 is 's


In [20]:
sen = "good morning sir fdsfa"
representation = []
for word in sen.split():
    if word not in embed_model.key_to_index:
        print(f"{word} not an existing word in the model")
        continue
    else:
        word_id = torch.tensor(embed_model.key_to_index[word])  # ID of the word in the embedding
    vec = embed_model[word]
    representation.append(vec)
    print(torch.equal(torch.Tensor(vec),embedding(word_id)))

representation = np.asarray(representation)
print(representation.shape)

True
True
True
fdsfa not an existing word in the model
(3, 300)


In [5]:
#Yuval - GRU
"""
class GRUNET(nn.Module):
    def __init__(self, input_size,hidden_size):
        super(GRUNET, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sigmoid=nn.Sigmoid()
        self.tanh=nn.Tanh()
        nn.init.kaiming_normal_(self.hidden.weight)

        #layers
        self.h = nn.Linear(self.hidden_size + self.input_size, self.hidden_size)
        self.reset=nn.Linear(self.hidden_size + self.input_size, self.hidden_size)
        self.update=nn.Linear(self.hidden_size + self.input_size, self.reset*self.hidden_size)
        self.h_tilda=nn.Linear(self.hidden_size + self.input_size, self.reset*self.hidden_size)
    

    def forward(self, x, input_embeddings):
    #initialize hidden output with zeroes
        z=self.update
        
        
        outputs,hidden_states=[],[]

        output=torch.zeros(self.hidden_state,dtype=torch.long)
        for val in input_embeddings:
        
            x=val
            x=torch.cat((val,output))

            x = self.hidden(x)
            hidden_states.append(x)

            x=nn.Sigmoid()
        
        outputs=torch.stack(outputs)
        hidden_states=torch.stack(hidden_states)
       
        out = output[-1, :, :]
        out = self.dropout(out)
        out = self.fc(out)
        return outputs, hidden_states
        """

In [21]:
class GRUCell(nn.Module):

    """
    An implementation of GRUCell.

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()



    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        
        x = x.view(-1, x.size(1))
        
        gate_x = self.x2h(x) 
        gate_h = self.h2h(hidden)
        
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()
        
        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_h.chunk(3, 1)
        
        
        resetgate = F.sigmoid(i_r + h_r)
        inputgate = F.sigmoid(i_i + h_i)
        newgate = F.tanh(i_n + (resetgate * h_n))
        
        hy = newgate + inputgate * (hidden - newgate)
        
        
        return hy

In [0]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(GRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
         
       
        self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
     
    
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        #print(x.shape,"x.shape")100, 28, 28
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
         
       
        outs = []
        
        hn = h0[0,:,:]
        
        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:], hn) 
            outs.append(hn)
            

        out = outs[-1].squeeze()
        
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out
 

In [31]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28
hidden_dim = 128
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 10
 
model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################
 
if torch.cuda.is_available():
    model.cuda()
     
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()
 
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

NameError: NameError: name 'GRUModel' is not defined

In [0]:
'''
STEP 7: TRAIN THE MODEL
'''
 
# Number of steps to unroll
seq_dim = 28 

loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
          
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
          
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        if torch.cuda.is_available():
            loss.cuda()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))